In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")

In [3]:
import certifi
os.environ['SSL_CERT_FILE'] = certifi.where()
from langchain_groq import ChatGroq
model = ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001DF865A7110>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001DF867E6D10>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))